In [ ]:
# Imports

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import models, transforms
from torchvision.transforms import InterpolationMode
import numpy as np
from typing import Tuple
import __main__
import requests

In [ ]:
#GPU 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
# Custom Dataset Class

class TaskDataset(Dataset):
    def __init__(self, transform=None):
        self.ids = []
        self.imgs = []
        self.labels = []
        self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)

class ImgLabelDataset(Dataset):
    def __init__(self, base_dataset):
        self.base = base_dataset

    def __len__(self):
        return len(self.base)

    def __getitem__(self, idx):
        sample = self.base[idx]
        return sample[1], sample[2]

In [ ]:
# Data Augmentation and Transforms

train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5)
])


val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

In [ ]:
torch.serialization.add_safe_globals([__main__.TaskDataset])

dataset = torch.load("Train.pt", weights_only=False)
print(f"Loaded dataset of length: {len(dataset)}")

# Split Datsaet into Train and Validation 90/10
dataset_size = len(dataset)
val_size = int(0.1 * dataset_size)
train_size = dataset_size - val_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform

Loaded dataset of length: 100000


In [ ]:
# DataLoaders

batch_size = 128
train_loader = DataLoader(ImgLabelDataset(train_dataset), batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(ImgLabelDataset(val_dataset), batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
# Define the Model - Resnet18
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# PGD (Projected Gradient Descent) attack implementation.
# Perturbs images within L-infinity ball of size eps.

def pgd_attack(model, images, labels, eps, alpha, iters, device='cuda'):
    model.eval()
    images, labels = images.to(device), labels.to(device)
    ori_images = images.clone().detach()
    perturbed_images = images + torch.empty_like(images).uniform_(-eps, eps)
    perturbed_images = torch.clamp(perturbed_images, 0, 1).detach()
    loss_fn = nn.CrossEntropyLoss()
    for _ in range(iters):
        perturbed_images.requires_grad = True
        outputs = model(perturbed_images)
        loss = loss_fn(outputs, labels)
        model.zero_grad()
        loss.backward()
        grad_sign = perturbed_images.grad.sign()
        adv_images = perturbed_images + alpha * grad_sign
        eta = torch.clamp(adv_images - ori_images, -eps, eps)
        perturbed_images = torch.clamp(ori_images + eta, 0, 1).detach()
    model.train()
    return perturbed_images

In [ ]:
# Train the model for one epoch on clean (unperturbed) data.

def train_one_epoch_clean(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

In [ ]:
# Train the model for one epoch using adversarial training.
# mix_clean=True: loss is averaged between clean and adversarial examples.

def train_one_epoch_adv(model, loader, optimizer, criterion, device,
                        eps=4/255, alpha=1/255, pgd_iters=7, mix_clean=True):
    model.train()
    running_loss = 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        adv_imgs = pgd_attack(model, imgs, labels, eps, alpha, pgd_iters, device)
        optimizer.zero_grad()
        if mix_clean:
            out_clean = model(imgs)
            out_adv = model(adv_imgs)
            loss = 0.5 * (criterion(out_clean, labels) + criterion(out_adv, labels))
        else:
            out_adv = model(adv_imgs)
            loss = criterion(out_adv, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

In [ ]:
# Evaluate the model. Returns accuracy on clean data, and on adversarial data if an attack function is provided.

def validate(model, loader, device, attack=None, eps=4/255, alpha=1/255, iters=7):
    model.eval()
    correct_clean = 0
    correct_adv = 0
    total = 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        total += labels.size(0)
        outputs = model(imgs)
        _, pred = outputs.max(1)
        correct_clean += pred.eq(labels).sum().item()
        if attack is not None:
            with torch.enable_grad():
                adv_imgs = attack(model, imgs, labels, eps, alpha, iters, device=device)
            outputs_adv = model(adv_imgs)
            _, pred_adv = outputs_adv.max(1)
            correct_adv += pred_adv.eq(labels).sum().item()
    clean_acc = 100 * correct_clean / total
    adv_acc = 100 * correct_adv / total if attack is not None else None
    return clean_acc, adv_acc

In [18]:
# ---- TRAINING FLOW ----
best_val_acc = 0
best_model_path = "submission_model01.pt"

In [ ]:
# Clean Training
for epoch in range(40):
    print(f"Epoch {epoch+1}/40 - Clean training")
    loss = train_one_epoch_clean(model, train_loader, optimizer, criterion, device)
    print(f"Loss: {loss:.4f}")
    val_acc, _ = validate(model, val_loader, device)
    print(f"Validation Accuracy: {val_acc:.2f}%")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"Saved best clean model with accuracy {best_val_acc:.2f}%")

Epoch 1/40 - Clean training
Loss: 1.3313
Validation Accuracy: 41.38%
Saved best clean model with accuracy 41.38%
Epoch 2/40 - Clean training
Loss: 1.1916
Validation Accuracy: 53.26%
Saved best clean model with accuracy 53.26%
Epoch 3/40 - Clean training
Loss: 1.1234
Validation Accuracy: 58.63%
Saved best clean model with accuracy 58.63%
Epoch 4/40 - Clean training
Loss: 1.0705
Validation Accuracy: 59.99%
Saved best clean model with accuracy 59.99%
Epoch 5/40 - Clean training
Loss: 1.0249
Validation Accuracy: 59.83%
Epoch 6/40 - Clean training
Loss: 0.9804
Validation Accuracy: 62.18%
Saved best clean model with accuracy 62.18%
Epoch 7/40 - Clean training
Loss: 0.9359
Validation Accuracy: 61.23%
Epoch 8/40 - Clean training
Loss: 0.8830
Validation Accuracy: 55.64%
Epoch 9/40 - Clean training
Loss: 0.8226
Validation Accuracy: 58.13%
Epoch 10/40 - Clean training
Loss: 0.7538
Validation Accuracy: 60.90%
Epoch 11/40 - Clean training
Loss: 0.6809
Validation Accuracy: 60.45%
Epoch 12/40 - Clean

In [ ]:
model.load_state_dict(torch.load(best_model_path))
optimizer = optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-4)

In [ ]:
# Adversarial Fine-Tuning

best_adv_val_acc = 0.0
best_adv_model_path = "best_adv_model01.pt"
num_adv_epochs = 15

for epoch in range(num_adv_epochs):
    print(f"Epoch {epoch+1}/{num_adv_epochs} - Adversarial fine-tuning")
    adv_loss = train_one_epoch_adv(model, train_loader, optimizer, criterion, device,
                                  eps=4/255, alpha=1/255, pgd_iters=7, mix_clean=True)
    print(f"Adv Train Loss: {adv_loss:.4f}")
    
    val_acc, adv_acc = validate(model, val_loader, device, attack=pgd_attack,
                               eps=4/255, alpha=1/255, iters=7)
    print(f"Validation Clean Accuracy: {val_acc:.2f}%")
    print(f"Validation Adversarial Accuracy: {adv_acc:.2f}%")
    
    # Save checkpoint if adversarial accuracy improved
    if adv_acc > best_adv_val_acc:
        best_adv_val_acc = adv_acc
        torch.save(model.state_dict(), best_adv_model_path)
        print(f"Saved best adversarial fine-tuned model at epoch {epoch+1} with adv accuracy {adv_acc:.2f}%")

Epoch 1/15 - Adversarial fine-tuning
Adv Train Loss: 1.6508
Validation Clean Accuracy: 53.39%
Validation Adversarial Accuracy: 33.65%
Saved best adversarial fine-tuned model at epoch 1 with adv accuracy 33.65%
Epoch 2/15 - Adversarial fine-tuning
Adv Train Loss: 1.4718
Validation Clean Accuracy: 54.06%
Validation Adversarial Accuracy: 36.53%
Saved best adversarial fine-tuned model at epoch 2 with adv accuracy 36.53%
Epoch 3/15 - Adversarial fine-tuning
Adv Train Loss: 1.4393
Validation Clean Accuracy: 54.55%
Validation Adversarial Accuracy: 37.31%
Saved best adversarial fine-tuned model at epoch 3 with adv accuracy 37.31%
Epoch 4/15 - Adversarial fine-tuning
Adv Train Loss: 1.4197
Validation Clean Accuracy: 54.82%
Validation Adversarial Accuracy: 38.04%
Saved best adversarial fine-tuned model at epoch 4 with adv accuracy 38.04%
Epoch 5/15 - Adversarial fine-tuning
Adv Train Loss: 1.4038
Validation Clean Accuracy: 54.57%
Validation Adversarial Accuracy: 38.76%
Saved best adversarial fin

In [ ]:
# Save final model for submission

print("Saving final model for submission...")
torch.save(model.state_dict(), "final_submission_model01.pt")

Saving final model for submission...


In [ ]:
# Submission

token = "53077688"  # your token here

with open("final_submission_model01.pt", "rb") as f:
    response = requests.post(
        "http://34.122.51.94:9090/robustness",
        files={"file": f},
        headers={"token": token, "model-name": "resnet18"}
    )


print(response.json())

c:\Users\maitr\miniconda3\envs\tf\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


{'clean_accuracy': 0.5746666666666667, 'fgsm_accuracy': 0.29633333333333334, 'pgd_accuracy': 0.10433333333333333}
